In [6]:
print("ok")

ok


In [7]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\ASUS\anaconda3\envs\adhdchatBot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:
PINECONE_API_KEY="80ffeaf1-be81-4e81-99ae-5ee13bf3e1fc"
PINECONE_API_ENV="gcp-starter"

In [11]:
#Extract data from pdf
def load_pdf(data):
    loader=DirectoryLoader(data,glob='*.pdf',loader_cls=PyMuPDFLoader)
    documents=loader.load()
    return documents


In [12]:
extractedData=load_pdf("dataSet/")

In [13]:
#Create text chunks
def text_split(extractedData):
    textSplitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    textChunks=textSplitter.split_documents(extractedData)
    return textChunks

In [14]:
textChunks=text_split(extractedData)

In [15]:
textChunks

[Document(page_content='Adult ADHD Self-Report Scale (ASRS-v1.1) Symptom Checklist \nInstructions \n \nThe questions on the back page are designed to stimulate dialogue between you and your patients and to help \nconfirm if they may be suffering from the symptoms of attention-deficit/hyperactivity disorder (ADHD). \n \nDescription:  The Symptom Checklist is an instrument consisting of the eighteen DSM-IV-TR criteria.  \nSix of the eighteen questions were found to be the most predictive of symptoms consistent with', metadata={'source': 'dataSet\\adhd-questionnaire-ASRS111.pdf', 'file_path': 'dataSet\\adhd-questionnaire-ASRS111.pdf', 'page': 0, 'total_pages': 3, 'format': 'PDF 1.5', 'title': '10852_elto_question_fhp2.PDF', 'author': 'shelbyw', 'subject': '', 'keywords': '', 'creator': 'QuarkXPress (tm) - [10852_elto_question_fhp2.qxd : 10852_elto_question_fhp2.qxd]', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'creationDate': 'D:20031003160125Z', 'modDate': "D:20110916085029-04'0

In [16]:
#downloading embedding model
def downloadHuggingFaceEmbeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings=downloadHuggingFaceEmbeddings()

In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [19]:
queryResult=embeddings.embed_query("Hi there!")
queryResult

[-0.09151122719049454,
 0.02514779008924961,
 0.002047134330496192,
 0.005181116051971912,
 -0.015277020633220673,
 -0.05271577835083008,
 0.016557365655899048,
 0.0423915795981884,
 -0.029215754941105843,
 0.0068251099437475204,
 -0.02768445573747158,
 0.007252746261656284,
 -0.01055943313986063,
 -0.04191434755921364,
 0.03230905532836914,
 0.03700309991836548,
 -0.05315249040722847,
 -0.00993194617331028,
 -0.10471194982528687,
 -0.039068449288606644,
 0.03552796319127083,
 0.07589998096227646,
 -0.10407808423042297,
 0.04460618272423744,
 -0.02690213918685913,
 -0.0319899246096611,
 -0.006866826210170984,
 0.06416881829500198,
 -0.0711832121014595,
 -0.017583850771188736,
 0.012554123066365719,
 0.12845946848392487,
 -0.022762032225728035,
 0.05372186005115509,
 0.029169755056500435,
 0.048878032714128494,
 -0.04694395512342453,
 -0.07912524789571762,
 0.0451648011803627,
 0.02378946915268898,
 -0.011888740584254265,
 0.011806312017142773,
 -0.0173427052795887,
 0.03931812942028046

In [20]:
from langchain.vectorstores import FAISS

In [21]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [22]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",config={'max_new_tokens':512,'temperature':0.8})

In [23]:
vectorstore=FAISS.from_texts(texts= [doc.page_content for doc in textChunks],embedding=embeddings)

In [24]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
def getConversationChain(vectorstore):
    memory=ConversationBufferMemory(memory_keys='chat_history',return_messages=True)
    conversationchain=ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversationchain

In [25]:
# Conversation Chain
conversation=getConversationChain(vectorstore)

In [26]:
prompt_tempate="""
Use the following pieces of information to aswer the user's question.
If you don't know the answer,just say you don't know,don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [27]:
prompt=PromptTemplate(template=prompt_tempate,input_variables=["context","question"])
chain_type_kwargs={"prompt":prompt}

In [47]:
def generate_response_with_llma(prompt, chain_type_kwargs={}):
  """
  Generates a response using LLaMA with the provided prompt and chain arguments.

  Args:
      prompt: The prompt string for LLaMA.
      chain_type_kwargs: Optional keyword arguments for specific chain types (if used).

  Returns:
      The generated response from LLaMA as a dictionary.
  """

  # Execute the prompt on LLaMA
  response = llm.(prompt, **chain_type_kwargs)
  return response

In [48]:
def handle_conversation(context, question):
  """
  Takes user input (context and question) and generates a response using LLaMA.

  Args:
      context: String containing relevant background information.
      question: User's question.

  Returns:
      A dictionary containing the generated response or an indication of no answer found.
  """
  prompt_tempate="""
    Use the following pieces of information to aswer the user's question.
    If you don't know the answer,just say you don't know,don't try to make up an answer.

    Context: {context}
    Question: {question}

    Only return the helpful answer below and nothing else.
    Helpful answer:
    """
  prompt =   prompt = f"""
  Use the following pieces of information to answer the user's question.
  If you don't know the answer, just say you don't know, don't try to make up an answer.

  Context: {context}
  Question: {question}

  Only return the helpful answer below and nothing else.
  Helpful answer:
  """
  chain_type_kwargs={"prompt":prompt}
  # Assuming you have a function to generate response using LLaMA with the prompt
  response = generate_response_with_llma(prompt, chain_type_kwargs)

  # Check if LLaMA generated a helpful answer
  if "helpful answer" in response:
    return {"answer": response["helpful answer"]}
  else:
    return {"answer": "I don't know the answer to that question yet."}


In [49]:
context = input("Provide some context (e.g., what were we talking about?): ")
question = input("Ask your question: ")

response = handle_conversation(context, question)
print(response)


ValueError: Argument 'prompts' is expected to be of type List[str], received argument of type <class 'str'>.

In [52]:
prompt=input()

In [53]:
query_answer=vectorstore.similarity_search(prompt)
query_answer[0].page_content

'diagnostic process?\nThe treatment questions were as\nfollows:\n1. What are the comparative safety\nand effectiveness of pharmacologic\nand/or nonpharmacologic\ntreatments of ADHD in improving\noutcomes associated with ADHD?\n2. What is the risk of diversion of\npharmacologic treatment?\n3. What are the comparative safety\nand effectiveness of different\nmonitoring strategies to evaluate\nthe effectiveness of treatment or\nchanges in ADHD status (eg,\nworsening or resolving\nsymptoms)?'

In [ ]:
retriever=vectorstore.as_retriever()
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retriever)
query="What is adhd ?"
results=qa.invoke(query)
results

AttributeError: 'RetrievalQA' object has no attribute 'invoke'